In [ ]:
import kagglehub
andrewmvd_helmet_detection_path = kagglehub.dataset_download('andrewmvd/helmet-detection')

print('Data source import complete.')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 391M/391M [00:32<00:00, 12.4MB/s] 

Extracting files...


Data source import complete.


In [2]:
import os
from pathlib import Path

# Get the download path
download_path = andrewmvd_helmet_detection_path
print("Dataset downloaded to:", download_path)

# To get absolute path
absolute_path = Path(download_path).resolve()
print("Absolute path:", absolute_path)

# To check if path exists and show contents
if Path(download_path).exists() and Path(download_path).is_dir():
    print("Contents:", os.listdir(download_path))

Dataset downloaded to: /Users/kushalgupta/.cache/kagglehub/datasets/andrewmvd/helmet-detection/versions/1
Absolute path: /Users/kushalgupta/.cache/kagglehub/datasets/andrewmvd/helmet-detection/versions/1
Contents: ['images', 'annotations']


In [ ]:
# Install required packages

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import torch
from PIL import Image
import xml.etree.ElementTree as ET
from lxml import etree

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Define paths
annotations_dir = r"/Users/kushalgupta/Desktop/DeepLearning/datasets/1/annotations"
images_dir = r"/Users/kushalgupta/Desktop/DeepLearning/datasets/1/images"



Using device: cpu


In [6]:
# Define classes
classes = ['With Helmet', 'Without Helmet']
class_dict = {'With Helmet': 0, 'Without Helmet': 1}

def parse_annotation(annotation_file):
    """Parse XML annotations to extract bounding boxes and labels"""
    tree = etree.parse(annotation_file)
    root = tree.getroot()
    objects = []
    for obj in root.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text), int(bbox.find('ymin').text),
                              int(bbox.find('xmax').text), int(bbox.find('ymax').text)]
        objects.append(obj_struct)
    return objects

def prepare_yolo_dataset(annotations_dir, images_dir, output_dir):
    """Prepare dataset in YOLO format"""
    # Create directories
    os.makedirs(os.path.join(output_dir, 'images', 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images', 'val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels', 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels', 'val'), exist_ok=True)
    
    annotation_files = [f for f in os.listdir(annotations_dir) if f.endswith('.xml')]
    train_files, val_files = train_test_split(annotation_files, test_size=0.2, random_state=42)
    
    def process_files(files, split):
        for annotation_file in files:
            # Parse annotation
            annotation_path = os.path.join(annotations_dir, annotation_file)
            objects = parse_annotation(annotation_path)
            
            # Image path
            image_path = os.path.join(images_dir, annotation_file.replace('.xml', '.png'))
            img = cv2.imread(image_path)
            if img is None:
                continue
                
            h, w = img.shape[:2]
            
            # Copy image
            img_filename = annotation_file.replace('.xml', '.png')
            cv2.imwrite(os.path.join(output_dir, 'images', split, img_filename), img)
            
            # Create YOLO format labels
            label_filename = annotation_file.replace('.xml', '.txt')
            label_path = os.path.join(output_dir, 'labels', split, label_filename)
            
            with open(label_path, 'w') as f:
                for obj in objects:
                    class_id = class_dict[obj['name']]
                    xmin, ymin, xmax, ymax = obj['bbox']
                    
                    # Convert to YOLO format (normalized)
                    x_center = (xmin + xmax) / 2 / w
                    y_center = (ymin + ymax) / 2 / h
                    width = (xmax - xmin) / w
                    height = (ymax - ymin) / h
                    
                    f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
    
    process_files(train_files, 'train')
    process_files(val_files, 'val')
    
    # Create dataset.yaml file
    yaml_content = f"""
path: {output_dir}
train: images/train
val: images/val

nc: {len(classes)}
names: {classes}
"""
    
    with open(os.path.join(output_dir, 'dataset.yaml'), 'w') as f:
        f.write(yaml_content)
    
    return os.path.join(output_dir, 'dataset.yaml')

!")

zsh:1: unmatched "


In [7]:
# Prepare dataset
output_dir = 'helmet_yolo_dataset'
yaml_path = prepare_yolo_dataset(annotations_dir, images_dir, output_dir)
print(f"Dataset prepared at: {output_dir}")

# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # Using nano version for faster training

# Train the model
results = model.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,
    device=str(device),
    patience=10,
    save=True,
    verbose=True
)

# Load the best model
best_model_path = 'runs/detect/train/weights/best.pt'
model = YOLO(best_model_path)

# Evaluate the model
metrics = model.val()
print("YOLOv8 Validation Metrics:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

# Test the model on sample images
def test_yolo_model(model, test_images_dir, num_images=5):
    """Test YOLOv8 model on sample images"""
    image_files = [f for f in os.listdir(test_images_dir) if f.endswith('.png')][:num_images]
    
    fig, axes = plt.subplots(1, num_images, figsize=(20, 4))
    if num_images == 1:
        axes = [axes]
    
    for i, img_file in enumerate(image_files):
        img_path = os.path.join(test_images_dir, img_file)
        
        # Perform detection
        results = model(img_path)
        
        # Plot results
        result_img = results[0].plot()
        result_img_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
        
        axes[i].imshow(result_img_rgb)
        axes[i].axis('off')
        axes[i].set_title(f'Detection: {img_file}')
    
    plt.tight_layout()
    plt.show()

# Test on validation images
test_images_dir = os.path.join(output_dir, 'images', 'val')
test_yolo_model(model, test_images_dir)

# Save YOLO model performance metrics
yolo_metrics = {
    'map50': metrics.box.map50,
    'map50_95': metrics.box.map,
    'precision': metrics.box.mp,
    'recall': metrics.box.mr
}

print("YOLOv8 Training Completed")

Dataset prepared at: helmet_yolo_dataset
WARNING ⚠️ Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov8n.pt... <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)>


######################################################################## 100.0%


Ultralytics 8.3.221 🚀 Python-3.11.9 torch-2.9.0 CPU (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=helmet_yolo_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=Tr

<Figure size 2000x400 with 5 Axes>

YOLOv8 Training Completed
